In [1]:
import numpy as np

def sample_numerical_kform_neighborhood_val(sampler, point, epsilon=1e-8):
    """
    Sample numerical values of a k-form at a (cubic) neighborhood of a point
    :param sampler: function to sample the k-form
    :param point: point at which to sample (center of a cube)
    :param epsilon: distance from the point to the neighborhood points
    :return: numerical values of the k-form at the neighborhood points as a dictionary
    6-dimensional dictionary with keys '0' to '6', where '0' is the center point and '1' to '6' are the -+ epsilon* canonical basis vectors 
    """
    p = np.array(point)
    directions = [np.array(np.eye(6, dtype=int)[i]) for i in range(6)]
    values = {"0": [sampler(p)],
            "1": [sampler(p - epsilon*directions[0]), sampler(p + epsilon*directions[0])],
            "2": [sampler(p - epsilon*directions[1]), sampler(p + epsilon*directions[1])],
            "3": [sampler(p - epsilon*directions[2]), sampler(p + epsilon*directions[2])],
            "4": [sampler(p - epsilon*directions[3]), sampler(p + epsilon*directions[3])],
            "5": [sampler(p - epsilon*directions[4]), sampler(p + epsilon*directions[4])],
            "6": [sampler(p - epsilon*directions[5]), sampler(p + epsilon*directions[5])],
            
            } 
            
    return values
    
def numerical_d(k_form_neighborhood_vals, e=1e-8):
    # d\omega_p(X_0, ..., X_k) \approx \frac{1}{\text{vol}(\sigma)} \int_{\partial \sigma} \omega 
    # where \sigma is a small k-dimensional block generated by X_0, ..., X_k
    # ----
    # Also, remember g_{ij}(x) = \delta_{ij} - \frac{1}{3} R_{ikjl}(0) \, x^k x^l + \mathcal{O}(|x|^3) = 
    #                          =\delta_{ij} + \mathcal{O}(|x|^3), 
    # since R_{ikjl} = 0 for Ricci-flat manifolds (which we assume here)
    # Thus, the volume of the k-dimensional block is approximately (2e)^k 
    # ----
    # Note for future Tomas: (-1)xSign convention of Acker's Book!
    """
    Compute the numerical exterior derivative of a k-form at a point using neighborhood values
    :param k_form_neighborhood_vals: dictionary of neighborhood values of the k-form
    :param g: metric tensor (needed for computing hypercube volume)
    :param e: small value to approximate the derivative
    :return: numerical exterior derivative of the k-form a the point k_form_neighborhood_vals["0"] as a numpy array
    """
    
    k = len(k_form_neighborhood_vals["0"][0].shape)
    
    # If the k-form is a 0-form (scalar field)
    if k == 0: 
        dw = np.zeros(6)
        for i in range(6):
            dw[i] = (1/(2*e))*(k_form_neighborhood_vals[str(i+1)][1] - k_form_neighborhood_vals[str(i+1)][0])
        return dw
    
    # If the k-form is a 1-form
    elif k == 1:
        dw = np.zeros((6,6))
        for i in range(6):
            for j in range(6):
                if i!=j:
                    dw[i,j] = (1/(2*e))*(k_form_neighborhood_vals[str(j+1)][0][i] +\
                                                k_form_neighborhood_vals[str(i+1)][1][j] -\
                                                k_form_neighborhood_vals[str(j+1)][1][i] -\
                                                k_form_neighborhood_vals[str(i+1)][0][j])
        
        return dw
    
    # If the k-form is a 2-form
    elif k == 2:
        dw = np.zeros((6,6,6))      
        for i in range(6):
            for j in range(6):
                for k in range(6):
                    if i!=j and j!=k and i!=k:
                        dw[i,j,k] = (1/(2*e))*(+k_form_neighborhood_vals[str(i+1)][1][j,k] -\
                                                k_form_neighborhood_vals[str(i+1)][0][j,k] -\
                                                k_form_neighborhood_vals[str(j+1)][1][i,k] +\
                                                k_form_neighborhood_vals[str(j+1)][0][i,k] +\
                                                k_form_neighborhood_vals[str(k+1)][1][i,j] -\
                                                k_form_neighborhood_vals[str(k+1)][0][i,j])
        return dw

    elif k >= 3:
        raise NotImplementedError("Numerical exterior derivative for k-forms with k >= 3 is not implemented.")

In [2]:
# The actual sampler will be a trained model that gets the kahler form at any point!!!
def sampler_2form(_):
    """
    Dummy sampler function for testing purposes.
    Replace this with the actual sampling function for your k-form.
    """
    return np.random.rand(6,6)  # Example: returns a random 6-dimensional vector

In [3]:
dic2_form = sample_numerical_kform_neighborhood_val(sampler_2form, [0, 0, 0, 0, 0, 0])
dic2_form

{'0': [array([[0.97251335, 0.66685882, 0.24879352, 0.92674503, 0.52617224,
          0.97016014],
         [0.37895815, 0.99020387, 0.20093996, 0.70434511, 0.23122691,
          0.22985789],
         [0.20536108, 0.33033923, 0.35061743, 0.41737379, 0.97018262,
          0.76648475],
         [0.99391547, 0.57742378, 0.06713953, 0.27049089, 0.14790629,
          0.91601839],
         [0.83375152, 0.05824376, 0.46521472, 0.37729647, 0.64088944,
          0.93353498],
         [0.91784498, 0.6110908 , 0.02707428, 0.80644569, 0.08347734,
          0.21754028]])],
 '1': [array([[0.26911765, 0.13825395, 0.22611798, 0.1203447 , 0.99403106,
          0.79701104],
         [0.57209276, 0.81525027, 0.72786315, 0.9406821 , 0.90237395,
          0.23748221],
         [0.68986131, 0.33484381, 0.38732838, 0.5445328 , 0.9650784 ,
          0.00341966],
         [0.07680373, 0.16019054, 0.15389396, 0.19127889, 0.73578785,
          0.23887782],
         [0.59608733, 0.1003845 , 0.70907379, 0.08888787,

In [4]:
# NB: you should pass a dict for the numerical_d function, sice it expects values in the neighborhood of the point
numerical_exterior_d = numerical_d(dic2_form)
numerical_exterior_d.shape #Returns (6, 6, 6), a 3-form at the point (0, 0, 0, 0, 0, 0)

(6, 6, 6)